# Market Analysis

Importing required packages

In [395]:
import pandas as pd
import numpy as np 
import plotly.express as px
import plotly.graph_objects as go

Getting the folder of the input files and defining local time

In [378]:
INPUT_FOLDER = "C:/Users/g.decertaines/Documents/GitHub/Project/Test_Market_Analysis/inputs"
LOCAL_TZ = "Europe/Brussels"

## 1. Preprocessing commodity price data

Importing commodity prices into dataframe

In [379]:
commodity = pd.read_csv(
    INPUT_FOLDER + "/commodity_prices.csv",
    names=["Date", "NCG_DAH", "EUA", "Coal_ARA"],
    parse_dates=["Date"],
    header=1)
commodity["Date"] = commodity["Date"].dt.tz_localize(LOCAL_TZ)
commodity = commodity.set_index("Date")

In [380]:
commodity.head()

,NCG_DAH,EUA,Coal_ARA
Date,,,
2020-01-02 00:00:00+01:00,12.400,24.24,6.572857
2020-01-03 00:00:00+01:00,12.018,24.86,6.795714
2020-01-04 00:00:00+01:00,13.113,NaN,NaN
2020-01-05 00:00:00+01:00,13.113,NaN,NaN
2020-01-06 00:00:00+01:00,13.212,24.13,6.795714


Checking if there are NA values

In [381]:
commodity.isna().sum()

NCG_DAH       0
EUA         369
Coal_ARA    387
dtype: int64

EUA and Coal prices are only published during week days hence the NA values. In order to keep consistency with week-end NG prices, I duplicated friday's EUA and Coal quotes over the next week-end

In [382]:
commodity = commodity.ffill()
commodity = commodity.round(2)
commodity.head()

,NCG_DAH,EUA,Coal_ARA
Date,,,
2020-01-02 00:00:00+01:00,12.40,24.24,6.57
2020-01-03 00:00:00+01:00,12.02,24.86,6.80
2020-01-04 00:00:00+01:00,13.11,24.86,6.80
2020-01-05 00:00:00+01:00,13.11,24.86,6.80
2020-01-06 00:00:00+01:00,13.21,24.13,6.80


Checking data consistency

In [383]:
commodity.describe().round(2)

,NCG_DAH,EUA,Coal_ARA
count,1.247000e+03,1247.00,1247.00
mean,3.444298e+06,57.08,20.05
std,1.216261e+08,25.60,15.04
min,3.580000e+00,15.23,4.88
25%,1.380000e+01,28.39,6.91
50%,4.008000e+01,58.96,16.81
75%,8.750000e+01,81.73,27.58
max,4.294967e+09,97.58,59.28


The max NG price is not consistent with actual NG prices. This outlier is removed from the dataset.

In [384]:
max = commodity["NCG_DAH"].max()
commodity = commodity[commodity["NCG_DAH"] < max]
commodity.describe().round(2)

,NCG_DAH,EUA,Coal_ARA
count,1246.00,1246.00,1246.00
mean,57.73,57.05,20.05
std,56.03,25.59,15.05
min,3.58,15.23,4.88
25%,13.78,28.38,6.90
50%,40.06,58.96,16.81
75%,87.34,81.73,27.60
max,319.56,97.58,59.28


Values seem to be consistent (max price of 315 €/MWh of natural gas was reached back in August 2022 following the Nord Stream 1 & 2 sabotage)

Importing power data into dataframe

In [385]:
power_columns = ["Hour",
            "PW_DAH_DE",
            "Wind_P10",
            "Wind_P50",
            "Wind_P90",
            "Solar_P10",
            "Solar_P50",
            "Solar_P90",
            "Lignite_Availability",
            "Nuclear_Availability",
            "Coal_Availability"]

power = pd.read_csv(
    INPUT_FOLDER + "/power_data.csv",
    names=power_columns,
    parse_dates=["Hour"],
    header=0)
power["Hour"] = power["Hour"].dt.tz_convert(LOCAL_TZ)
power = power.set_index("Hour")


In [387]:
power.isna().sum()

PW_DAH_DE               0
Wind_P10                0
Wind_P50                0
Wind_P90                0
Solar_P10               0
Solar_P50               0
Solar_P90               0
Lignite_Availability    4
Nuclear_Availability    4
Coal_Availability       4
dtype: int64

The time change each March is creating NA values in the availability dataset. I propose to duplicate the values from the past hour.

In [388]:
power = power.ffill()

In [389]:
power.describe().round(2)

,PW_DAH_DE,Wind_P10,Wind_P50,Wind_P90,Solar_P10,Solar_P50,Solar_P90,Lignite_Availability,Nuclear_Availability,Coal_Availability
count,29929.00,29929.00,29929.00,29929.00,29929.00,29929.00,29929.00,29929.00,29929.00,29929.00
mean,119.02,13085.32,14339.30,15828.37,5375.07,5662.34,5930.07,13909.15,5508.35,9130.45
std,119.76,10148.73,10750.37,11422.73,8175.43,8540.97,8879.97,1511.63,2715.92,2820.47
min,-129.96,128.70,156.86,239.92,0.00,0.00,0.00,7483.00,0.00,3103.10
25%,37.32,5089.13,5839.24,6748.23,0.00,0.00,0.00,12968.00,4056.00,7083.10
50%,78.31,10125.66,11267.04,12620.20,293.00,320.00,342.00,14020.00,6704.00,8859.24
75%,164.23,18807.38,20503.87,22551.06,8465.00,9009.00,9531.00,14975.00,8114.00,10978.24
max,871.00,50342.11,51111.33,51515.70,39527.00,40175.00,40861.00,16973.00,8114.00,15578.04


Similarly to gas prices, power prices reached close to 871 €/MWh during the energy crisis. 


In order to merge both commodity and power dataframes, I shall first resample both dataframe to hourly granularity

In [390]:
commodity = commodity.resample("H").ffill()

In [391]:
dataset = commodity.join(power)

In [394]:
dataset.head().round(2)

,NCG_DAH,EUA,Coal_ARA,PW_DAH_DE,Wind_P10,Wind_P50,Wind_P90,Solar_P10,Solar_P50,Solar_P90,Lignite_Availability,Nuclear_Availability,Coal_Availability
Date,,,,,,,,,,,,,
2020-01-02 00:00:00+01:00,12.4,24.24,6.57,35.40,10494.82,11402.40,12472.88,0.0,0.0,0.0,14739.0,8114.0,13792.94
2020-01-02 01:00:00+01:00,12.4,24.24,6.57,31.98,9748.15,10298.70,10982.42,0.0,0.0,0.0,14739.0,8114.0,13792.94
2020-01-02 02:00:00+01:00,12.4,24.24,6.57,30.50,9474.90,9996.68,10677.80,0.0,0.0,0.0,14739.0,8114.0,13777.94
2020-01-02 03:00:00+01:00,12.4,24.24,6.57,28.79,9385.93,9991.21,10757.49,0.0,0.0,0.0,14739.0,8114.0,13777.94
2020-01-02 04:00:00+01:00,12.4,24.24,6.57,28.42,9485.31,10257.12,11321.08,0.0,0.0,0.0,14739.0,8114.0,13777.94


# 2. Data Analysis